In [ ]:
import os
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import splitfolders
import cv2
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam


In [ ]:
input_folder = 'customdataset3'
output_folder = 'imagerecognition3'

split_ratio = (0.8,0.1,0.1)

splitfolders.ratio(

    input_folder,
    output=output_folder,
    seed=500,
    ratio = split_ratio,
    group_prefix = None
)

In [ ]:
img_size = (224,224)
batch_size = 32 # Model updates weights after processing 32 images

train_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode='nearest'
)

In [ ]:
# Rescale pixel values, without any extra augmentation processes for test and validation data

test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)
valid_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

In [ ]:
train_dir = os.path.join(output_folder, 'train')
val_dir = os.path.join(output_folder, 'val')
test_dir = os.path.join(output_folder, 'test')

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size = img_size,
    batch_size = batch_size,
    class_mode = 'categorical'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size = img_size,
    batch_size = batch_size,
    class_mode = 'categorical'
)

valid_data = valid_datagen.flow_from_directory(
    val_dir,
    target_size = img_size,
    batch_size = batch_size,
    class_mode = 'categorical'
)

In [ ]:
images, labels = next(valid_data)
idx = random.randint(0,images.shape[0]-1)

plt.imshow(images[idx])
plt.show()

In [ ]:
from keras.applications.resnet import ResNet50
base_model = ResNet50(weights='imagenet',include_top=False, input_shape=(img_size[0], img_size[1], 3))

# base_model.trainable = False # Only trains newly added classification layers

for layer in base_model.layers[:-30]:
    layer.trainable = False
for layer in base_model.layers[-30:]:
    layer.trainable = True
                      

In [ ]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')


])

In [ ]:
model.compile(
    optimizer = Adam(learning_rate=1e-5), #lower learning rate for fine tuning
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'])

In [ ]:
model.fit(train_data, epochs = 50, validation_data = valid_data)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
class_names = {0: 'eyesclosed',
               1: 'scream',
               2: 'tongue_out'
               }

In [ ]:
def predict_img(image,model):
    test_img=cv2.imread(image)
    test_img=cv2.resize(test_img,(224,224))
    test_img=np.expand_dims(test_img,axis=0)
    result=model.predict(test_img)
    r = np.argmax(result)
    print(class_names[r])

In [ ]:
predict_img('customdataset/tongue/tongue_2.jpeg',model)

In [ ]:
model.save('CustomNetworkv3.keras')